# Protein-ligand complex MD Setup tutorial using BioExcel Building Blocks (biobb) 
### --***AmberTools package version***--
**Based on the [MDWeb](http://mmb.irbbarcelona.org/MDWeb2/) [Amber FULL MD Setup tutorial](https://mmb.irbbarcelona.org/MDWeb2/help.php?id=workflows#AmberWorkflowFULL)**

***
This tutorial aims to illustrate the process of **setting up a simulation system** containing a **protein in complex with a ligand**, step by step, using the **BioExcel Building Blocks library (biobb)** wrapping the **AmberTools** utility from the **AMBER package**. The particular example used is the **T4 lysozyme** protein (PDB code [3HTB](https://www.rcsb.org/structure/3HTB)) with two residue modifications ***L99A/M102Q*** complexed with the small ligand **2-propylphenol** (3-letter code [JZ4](https://www.rcsb.org/ligand/JZ4)). 
***

## Settings

### Biobb modules used

 - [biobb_io](https://github.com/bioexcel/biobb_io): Tools to fetch biomolecular data from public databases.
 - [biobb_amber](https://github.com/bioexcel/biobb_amber): Tools to setup and run Molecular Dynamics simulations with AmberTools.
 - [biobb_structure_utils](https://github.com/bioexcel/biobb_structure_utils): Tools to modify or extract information from a PDB structure file.
 - [biobb_analysis](https://github.com/bioexcel/biobb_analysis): Tools to analyse Molecular Dynamics trajectories.
 - [biobb_chemistry](https://github.com/bioexcel/biobb_chemistry): Tools to to perform chemical conversions.
 
### Auxiliar libraries used

 - [nb_conda_kernels](https://github.com/Anaconda-Platform/nb_conda_kernels): Enables a Jupyter Notebook or JupyterLab application in one conda environment to access kernels for Python, R, and other languages found in other environments.
 - [nglview](http://nglviewer.org/#nglview): Jupyter/IPython widget to interactively view molecular structures and trajectories in notebooks.
 - [ipywidgets](https://github.com/jupyter-widgets/ipywidgets): Interactive HTML widgets for Jupyter notebooks and the IPython kernel.
 - [plotly](https://plot.ly/python/offline/): Python interactive graphing library integrated in Jupyter notebooks.
 - [simpletraj](https://github.com/arose/simpletraj): Lightweight coordinate-only trajectory reader based on code from GROMACS, MDAnalysis and VMD.

### Conda Installation and Launch

```console
git clone https://github.com/bioexcel/biobb_wf_md_setup_amber.git
cd biobb_wf_md_setup_amber
conda env create -f conda_env/environment.yml
conda activate biobb_MDsetup_tutorials_amber
jupyter-nbextension enable --py --user widgetsnbextension
jupyter-nbextension enable --py --user nglview
jupyter-notebook biobb_wf_md_setup_amber/notebooks/mdsetup_lig/biobb_amber_complex_setup_notebook.ipynb
``` 

***
## Pipeline steps
 1. [Input Parameters](#input)
 2. [Fetching PDB Structure](#fetch)
 3. [Preparing PDB file for AMBER](#pdb4amber)
 4. [Create ligand system topology](#ligtop)
 5. [Create Protein-Ligand Complex System Topology](#top)
 6. [Energetically Minimize the Structure](#minv)
 7. [Create Solvent Box and Solvating the System](#box)
 8. [Adding Ions](#ions)
 9. [Energetically Minimize the System](#min)
 10. [Heating the System](#heating)
 11. [Equilibrate the System (NVT)](#nvt)
 12. [Equilibrate the System (NPT)](#npt)
 13. [Free Molecular Dynamics Simulation](#free)
 14. [Post-processing and Visualizing Resulting 3D Trajectory](#post)
 15. [Output Files](#output)
 16. [Questions & Comments](#questions)
 
***
<img src="https://bioexcel.eu/wp-content/uploads/2019/04/Bioexcell_logo_1080px_transp.png" alt="Bioexcel2 logo"
	title="Bioexcel2 logo" width="400" />
***


<a id="input"></a>
## Input parameters
**Input parameters** needed:
 - **pdbCode**: PDB code of the protein structure (e.g. 3HTB)
 - **ligandCode**: 3-letter code of the ligand (e.g. JZ4)
 - **mol_charge**: Charge of the ligand (e.g. 0)

In [1]:
import nglview
import ipywidgets
import plotly
from plotly import subplots
import plotly.graph_objs as go

pdbCode = "3htb"
ligandCode = "JZ4"
mol_charge = 0

<a id="fetch"></a>
***
## Fetching PDB structure
Downloading **PDB structure** with the **protein molecule** from the RCSB PDB database.<br>
Alternatively, a **PDB file** can be used as starting structure. <br>
Stripping from the **downloaded structure** any **crystallographic water** molecule or **heteroatom**. <br>
***
**Building Blocks** used:
 - [pdb](https://biobb-io.readthedocs.io/en/latest/api.html#module-api.pdb) from **biobb_io.api.pdb**
 - [remove_pdb_water](https://biobb-structure-utils.readthedocs.io/en/latest/utils.html#module-utils.remove_pdb_water) from **biobb_structure_utils.utils.remove_pdb_water**
 - [remove_ligand](https://biobb-structure-utils.readthedocs.io/en/latest/utils.html#module-utils.remove_ligand) from **biobb_structure_utils.utils.remove_ligand**
***

In [2]:
# Import module
from biobb_io.api.pdb import pdb

# Create properties dict and inputs/outputs
downloaded_pdb = pdbCode+'.pdb'

prop = {
    'pdb_code': pdbCode,
    'filter': False
}

#Create and launch bb
pdb(output_pdb_path=downloaded_pdb,
    properties=prop)

2021-06-22 14:33:14,463 [MainThread  ] [INFO ]  Downloading: 3htb from: https://www.ebi.ac.uk/pdbe/entry-files/download/pdb3htb.ent
/anaconda3/envs/biobb_AMBER_MDsetup_tutorials/lib/python3.7/site-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ebi.ac.uk'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
2021-06-22 14:33:14,889 [MainThread  ] [INFO ]  Writting pdb to: 3htb.pdb


0

In [54]:
# Show protein
view = nglview.show_structure_file(downloaded_pdb)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view.add_representation(repr_type='ball+stick', radius='0.1', selection='water')
view.add_representation(repr_type='ball+stick', radius='0.5', selection='ligand')
view.add_representation(repr_type='ball+stick', radius='0.5', selection='ion')
view._remote_call('setSize', target='Widget', args=['','600px'])

view.render_image()
view.download_image(filename='ngl1.png')

view

NGLWidget()

<img src='ngl1.png'></img>

In [4]:
# Import module
from biobb_structure_utils.utils.remove_pdb_water import remove_pdb_water

# Create properties dict and inputs/outputs
nowat_pdb = pdbCode+'.nowat.pdb'

#Create and launch bb
remove_pdb_water(input_pdb_path=downloaded_pdb,
    output_pdb_path=nowat_pdb)

2021-06-22 14:33:16,315 [MainThread  ] [INFO ]  check_structure -i 3htb.pdb -o 3htb.nowat.pdb --force_save water --remove yes

2021-06-22 14:33:16,317 [MainThread  ] [INFO ]  Exit code 0

2021-06-22 14:33:16,318 [MainThread  ] [INFO ]  ===============================================================================
=                   BioBB structure checking utility v3.7.2                   =
=                 A. Hospital, P. Andrio, J.L. Gelpi 2018-20                  =

Structure 3htb.pdb loaded
 Title: 2-propylphenol in complex with t4 lysozyme l99a/m102q
 Experimental method: x-ray diffraction
 Keywords: hydrolase,glycosidase, bacteriolytic enzyme, antimicrobial
 Resolution: 1.81 A

 Num. models: 1
 Num. chains: 1 (A: Protein)
 Num. residues:  387
 Num. residues with ins. codes:  0
 Num. HETATM residues:  224
 Num. ligands or modified residues:  4
 Num. water mol.:  220
 Num. atoms:  1544
Small mol ligands found
PO4 A165
PO4 A166
JZ4 A167
BME A168

Running water. Options: --remove 

0

In [5]:
# Import module
from biobb_structure_utils.utils.remove_ligand import remove_ligand

# Removing PO4 ligands:

# Create properties dict and inputs/outputs
nopo4_pdb = pdbCode+'.noPO4.pdb'

prop = {
    'ligand' : 'PO4'
}

#Create and launch bb
remove_ligand(input_structure_path=nowat_pdb,
    output_structure_path=nopo4_pdb,
    properties=prop)

# Removing BME ligand:

# Create properties dict and inputs/outputs
nobme_pdb = pdbCode+'.noBME.pdb'

prop = {
    'ligand' : 'BME'
}

#Create and launch bb
remove_ligand(input_structure_path=nopo4_pdb,
    output_structure_path=nobme_pdb,
    properties=prop)

2021-06-22 14:33:16,341 [MainThread  ] [INFO ]  PDB format detected, removing all atoms from residues named PO4
2021-06-22 14:33:16,349 [MainThread  ] [INFO ]  PDB format detected, removing all atoms from residues named BME


0

## Visualizing 3D structure

In [55]:
# Show protein
view = nglview.show_structure_file(nobme_pdb)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view.add_representation(repr_type='ball+stick', radius='0.5', selection='hetero')
view._remote_call('setSize', target='Widget', args=['','600px'])

view.render_image()
view.download_image(filename='ngl2.png')

view

NGLWidget()

<img src='ngl2.png'></img>

<a id="pdb4amber"></a>
***
## Preparing PDB file for AMBER
Before starting a **protein MD setup**, it is always strongly recommended to take a look at the initial structure and try to identify important **properties** and also possible **issues**. These properties and issues can be serious, as for example the definition of **disulfide bridges**, the presence of a **non-standard aminoacids** or **ligands**, or **missing residues**. Other **properties** and **issues** might not be so serious, but they still need to be addressed before starting the **MD setup process**. **Missing hydrogen atoms**, presence of **alternate atomic location indicators** or **inserted residue codes** (see [PDB file format specification](https://www.wwpdb.org/documentation/file-format-content/format33/sect9.html#ATOM)) are examples of these not so crucial characteristics. Please visit the [AMBER tutorial: Building Protein Systems in Explicit Solvent](http://ambermd.org/tutorials/basic/tutorial7/index.php) for more examples. **AmberTools** utilities from **AMBER MD package** contain a tool able to analyse **PDB files** and clean them for further usage, especially with the **AmberTools LEaP program**: the **pdb4amber tool**. The next step of the workflow is running this tool to analyse our **input PDB structure**.<br>

For the particular **T4 Lysosyme** example, the most important property that is identified by the **pdb4amber** utility is the presence of **disulfide bridges** in the structure. Those are marked changing the residue names **from CYS to CYX**, which is the code that **AMBER force fields** use to distinguish between cysteines forming or not forming **disulfide bridges**. This will be used in the following step to correctly form a **bond** between these cysteine residues. 

We invite you to check what the tool does with different, more complex structures (e.g. PDB code [6N3V](https://www.rcsb.org/structure/6N3V)). 

***
**Building Blocks** used:
 - [pdb4amber](https://biobb-amber.readthedocs.io/en/latest/pdb4amber.html#module-pdb4amber.pdb4amber) from **biobb_amber.pdb4amber.pdb4amber**
***

In [7]:
# Import module
from biobb_amber.pdb4amber.pdb4amber import pdb4amber

# Create prop dict and inputs/outputs
output_pdb4amber_path = 'structure.pdb4amber.pdb'

# Create and launch bb
pdb4amber(input_pdb_path=nobme_pdb,
            output_pdb_path=output_pdb4amber_path,
            properties=prop)

/anaconda3/envs/biobb_AMBER_MDsetup_tutorials/lib/python3.7/site-packages/biobb_common/tools/file_utils.py:354: UserWarning: Warning: ligand is not a recognized property. The most similar property is: out_log
  error_property, close_property))
2021-06-22 14:33:16,599 [MainThread  ] [INFO ]  Creating bdfb3702-88bc-4dcc-82dd-51082d0619f9 temporary folder
2021-06-22 14:33:16,603 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-06-22 14:33:17,750 [MainThread  ] [INFO ]  pdb4amber  -i 3htb.noBME.pdb -o structure.pdb4amber.pdb

2021-06-22 14:33:17,752 [MainThread  ] [INFO ]  Exit code 0

2021-06-22 14:33:17,754 [MainThread  ] [INFO ]  
Summary of pdb4amber for: 3htb.noBME.pdb

----------Chains
The following (original) chains have been found:
A

---------- Alternate Locations (Original Residues!))

The following residues had alternate locations:
MET_1
THR_21
TYR_24
ASN_68
ASP_72
ARG_80
ARG_96
GLN_102
LYS_147
ARG_154
-----------Non-standard-resnames
J

0

<a id="ligtop"></a>
***
## Create ligand system topology
**Building AMBER topology** corresponding to the ligand structure.<br>
Force field used in this tutorial step is **amberGAFF**: [General AMBER Force Field](http://ambermd.org/antechamber/gaff.html), designed for rational drug design.<br>
- [Step 1](#ligandTopologyStep1): Extract **ligand structure**.
- [Step 2](#ligandTopologyStep2): Add **hydrogen atoms** if missing.
- [Step 3](#ligandTopologyStep3): **Energetically minimize the system** with the new hydrogen atoms. 
- [Step 4](#ligandTopologyStep4): Generate **ligand topology** (parameters). 
***
**Building Blocks** used:
 - [ExtractHeteroAtoms](https://biobb-structure-utils.readthedocs.io/en/latest/utils.html#module-utils.extract_heteroatoms) from **biobb_structure_utils.utils.extract_heteroatoms**
 - [ReduceAddHydrogens](https://biobb-chemistry.readthedocs.io/en/latest/ambertools.html#module-ambertools.reduce_add_hydrogens) from **biobb_chemistry.ambertools.reduce_add_hydrogens**
 - [BabelMinimize](https://biobb-chemistry.readthedocs.io/en/latest/babelm.html#module-babelm.babel_minimize) from **biobb_chemistry.babelm.babel_minimize** 
 - [AcpypeParamsAC](https://biobb-chemistry.readthedocs.io/en/latest/acpype.html#module-acpype.acpype_params_ac) from **biobb_chemistry.acpype.acpype_params_ac** 
***

<a id="ligandTopologyStep1"></a>
### Step 1: Extract **Ligand structure**

In [8]:
# Create Ligand system topology, STEP 1
# Extracting Ligand JZ4
# Import module
from biobb_structure_utils.utils.extract_heteroatoms import extract_heteroatoms

# Create properties dict and inputs/outputs
ligandFile = ligandCode+'.pdb'

prop = {
     'heteroatoms' : [{"name": "JZ4"}]
}

extract_heteroatoms(input_structure_path=output_pdb4amber_path,
     output_heteroatom_path=ligandFile,
     properties=prop)

2021-06-22 14:33:17,826 [MainThread  ] [INFO ]  File JZ4.pdb created


0

<a id="ligandTopologyStep2"></a>
### Step 2: Add **hydrogen atoms**

In [9]:
# Create Ligand system topology, STEP 2
# Reduce_add_hydrogens: add Hydrogen atoms to a small molecule (using Reduce tool from Ambertools package)
# Import module
from biobb_chemistry.ambertools.reduce_add_hydrogens import reduce_add_hydrogens

# Create prop dict and inputs/outputs
output_reduce_h = ligandCode+'.reduce.H.pdb' 

prop = {
    'nuclear' : 'true'
}

# Create and launch bb
reduce_add_hydrogens(input_path=ligandFile,
                   output_path=output_reduce_h,
                   properties=prop)

2021-06-22 14:33:17,857 [MainThread  ] [INFO ]  Not using any container
2021-06-22 14:33:17,972 [MainThread  ] [INFO ]  reduce -NUClear -OH -ROTNH3 -ALLALT JZ4.pdb > JZ4.reduce.H.pdb

2021-06-22 14:33:17,973 [MainThread  ] [INFO ]  Exit code 0

2021-06-22 14:33:17,974 [MainThread  ] [INFO ]  reduce: version 3.3 06/02/2016, Copyright 1997-2016, J. Michael Word
Processing file: "JZ4.pdb"
Database of HETATM connections: "/anaconda3/envs/biobb_AMBER_MDsetup_tutorials//dat/reduce_wwPDB_het_dict.txt"
VDW dot density = 16/A^2
Orientation penalty scale = 1 (100%)
Eliminate contacts within 3 bonds.
Ignore atoms with |occupancy| <= 0.01 during adjustments.
Waters ignored if B-Factor >= 40 or |occupancy| < 0.66
Aromatic rings in amino acids accept hydrogen bonds.
Building or keeping OH & SH Hydrogens.
Rotating NH3 Hydrogens.
Not processing Met methyls.
 Singles(size 1): A 164 JZ4 OAB 
 orientation 1: A 164 JZ4 OAB :   rot  120: bump=0.000, HB=0.000, total=0.000
Found 0 hydrogens (0 hets)
Standard

0

<a id="ligandTopologyStep3"></a>
### Step 3: **Energetically minimize the system** with the new hydrogen atoms. 

In [10]:
# Create Ligand system topology, STEP 3
# Babel_minimize: Structure energy minimization of a small molecule after being modified adding hydrogen atoms
# Import module
from biobb_chemistry.babelm.babel_minimize import babel_minimize

# Create prop dict and inputs/outputs
output_babel_min = ligandCode+'.H.min.mol2'   

prop = {
    'method' : 'sd',
    'criteria' : '1e-10',
    'force_field' : 'GAFF'
}


# Create and launch bb
babel_minimize(input_path=output_reduce_h,
              output_path=output_babel_min,
              properties=prop)

2021-06-22 14:33:17,994 [MainThread  ] [INFO ]  Hydrogens  is not correct, assigned default value: False
2021-06-22 14:33:17,996 [MainThread  ] [INFO ]  Steps  is not correct, assigned default value: 2500
2021-06-22 14:33:17,996 [MainThread  ] [INFO ]  Cut-off  is not correct, assigned default value: False
2021-06-22 14:33:17,997 [MainThread  ] [INFO ]  Rvdw  is not correct, assigned default value: 6.0
2021-06-22 14:33:17,998 [MainThread  ] [INFO ]  Rele  is not correct, assigned default value: 10.0
2021-06-22 14:33:17,999 [MainThread  ] [INFO ]  Frequency  is not correct, assigned default value: 10
2021-06-22 14:33:18,000 [MainThread  ] [INFO ]  Not using any container
2021-06-22 14:33:18,620 [MainThread  ] [INFO ]  obminimize -c 1e-10 -sd -ff GAFF -ipdb JZ4.reduce.H.pdb -omol2 > JZ4.H.min.mol2

2021-06-22 14:33:18,622 [MainThread  ] [INFO ]  Exit code 0

2021-06-22 14:33:18,623 [MainThread  ] [INFO ]  
A T O M   T Y P E S

IDX	TYPE	RING
1	c3	NO
2	ca	AR
3	ca	AR
4	ca	AR
5	ca	AR
6	ca	AR

0

### Visualizing 3D structures
Visualizing the small molecule generated **PDB structures** using **NGL**:  
- **Original Ligand Structure** (Left)
- **Ligand Structure with hydrogen atoms added** (with Reduce program) (Center)
- **Ligand Structure with hydrogen atoms added** (with Reduce program), **energy minimized** (with Open Babel) (Right) 

In [56]:
# Show different structures generated (for comparison)

view1 = nglview.show_structure_file(ligandFile)
view1.add_representation(repr_type='ball+stick')
view1._remote_call('setSize', target='Widget', args=['350px','400px'])
view1.camera='orthographic'

view1.render_image()
view1.download_image(filename='ngl3.png')

view1
view2 = nglview.show_structure_file(output_reduce_h)
view2.add_representation(repr_type='ball+stick')
view2._remote_call('setSize', target='Widget', args=['350px','400px'])
view2.camera='orthographic'

view2.render_image()
view2.download_image(filename='ngl4.png')

view2
view3 = nglview.show_structure_file(output_babel_min)
view3.add_representation(repr_type='ball+stick')
view3._remote_call('setSize', target='Widget', args=['350px','400px'])
view3.camera='orthographic'

view3.render_image()
view3.download_image(filename='ngl5.png')

view3
ipywidgets.HBox([view1, view2, view3])

<img src='ngl3.png' style='float:left;width:33%;' />
<img src='ngl4.png' style='float:left;width:33%;' />
<img src='ngl5.png' style='float:left;width:33%;' />

<a id="ligandTopologyStep4"></a>
### Step 4: Generate **ligand topology** (parameters).

In [12]:
# Create Ligand system topology, STEP 4
# Acpype_params_gmx: Generation of topologies for AMBER with ACPype
# Import module
from biobb_chemistry.acpype.acpype_params_ac import acpype_params_ac

# Create prop dict and inputs/outputs
output_acpype_inpcrd = ligandCode+'params.inpcrd'
output_acpype_frcmod = ligandCode+'params.frcmod'
output_acpype_lib = ligandCode+'params.lib'
output_acpype_prmtop = ligandCode+'params.prmtop'
output_acpype = ligandCode+'params'

prop = {
    'basename' : output_acpype,
    'charge' : mol_charge
}

# Create and launch bb
acpype_params_ac(input_path=output_babel_min, 
                output_path_inpcrd=output_acpype_inpcrd,
                output_path_frcmod=output_acpype_frcmod,
                output_path_lib=output_acpype_lib,
                output_path_prmtop=output_acpype_prmtop,
                properties=prop)

2021-06-22 14:33:18,871 [MainThread  ] [INFO ]  Running acpype, this execution can take a while
2021-06-22 14:33:18,872 [MainThread  ] [INFO ]  Not using any container
2021-06-22 14:33:21,690 [MainThread  ] [INFO ]  acpype -i /home/gbayarri_local/projects/BioBB/tutorials/biobb_wf_amber_md_setup/biobb_wf_amber_md_setup/notebooks/mdsetup_lig/JZ4.H.min.mol2 -b JZ4params.ZW79gA -n 0

2021-06-22 14:33:21,692 [MainThread  ] [INFO ]  Exit code 0

2021-06-22 14:33:21,692 [MainThread  ] [INFO ]  ========================================================================================
| ACPYPE: AnteChamber PYthon Parser interfacE v. 2019-11-07T23:16:00CET (c) 2021 AWSdS |
==> ... charge set to 0
==> Executing Antechamber...
==> * Antechamber OK *
==> * Parmchk OK *
==> Executing Tleap...
++++++++++start_quote+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Checking 'JZ4'....
Checking parameters for unit 'JZ4'.
Checking for bond parameters.
Checking for angle parameters.
Unit is OK.
+++

0

<a id="top"></a>
***
## Create protein-ligand complex system topology
**Building AMBER topology** corresponding to the protein-ligand complex structure.<br>

*IMPORTANT: the previous pdb4amber building block is changing the proper cysteines residue naming in the PDB file from CYS to CYX so that this step can automatically identify and add the disulfide bonds to the system topology.*<br>

The **force field** used in this tutorial is [**ff14SB**](https://doi.org/10.1021/acs.jctc.5b00255) for the **protein**, an evolution of the **ff99SB** force field with improved accuracy of protein side chains and backbone parameters; and the [**gaff**](https://doi.org/10.1002/jcc.20035) force field for the small molecule. **Water** molecules type used in this tutorial is [**tip3p**](https://doi.org/10.1021/jp003020w).<br>
Adding **side chain atoms** and **hydrogen atoms** if missing. Forming **disulfide bridges** according to the info added in the previous step. <br>

*NOTE: From this point on, the **protein-ligand complex structure and topology** generated can be used in a regular MD setup.*

Generating three output files: 
- **AMBER structure** (PDB file)
- **AMBER topology** (AMBER [Parmtop](https://ambermd.org/FileFormats.php#topology) file)
- **AMBER coordinates** (AMBER [Coordinate/Restart](https://ambermd.org/FileFormats.php#restart) file) 
***
**Building Blocks** used:
 - [leap_gen_top](https://biobb-amber.readthedocs.io/en/latest/leap.html#module-leap.leap_gen_top) from **biobb_amber.leap.leap_gen_top**
***

In [13]:
# Import module
from biobb_amber.leap.leap_gen_top import leap_gen_top

# Create prop dict and inputs/outputs
output_pdb_path = 'structure.leap.pdb'
output_top_path = 'structure.leap.top'
output_crd_path = 'structure.leap.crd'

prop = {
    "forcefield" : ["protein.ff14SB","gaff"]
}

# Create and launch bb
leap_gen_top(input_pdb_path=output_pdb4amber_path,
           input_lib_path=output_acpype_lib,
           input_frcmod_path=output_acpype_frcmod,
           output_pdb_path=output_pdb_path,
           output_top_path=output_top_path,
           output_crd_path=output_crd_path,
           properties=prop)

2021-06-22 14:33:21,724 [MainThread  ] [INFO ]  Creating 47a0068a-6a8f-44cf-b009-eb08c11b5da8 temporary folder
2021-06-22 14:33:21,725 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-06-22 14:33:22,111 [MainThread  ] [INFO ]  tleap  -f 47a0068a-6a8f-44cf-b009-eb08c11b5da8/leap.in

2021-06-22 14:33:22,113 [MainThread  ] [INFO ]  Exit code 0

2021-06-22 14:33:22,115 [MainThread  ] [INFO ]  -I: Adding /anaconda3/envs/biobb_AMBER_MDsetup_tutorials/dat/leap/prep to search path.
-I: Adding /anaconda3/envs/biobb_AMBER_MDsetup_tutorials/dat/leap/lib to search path.
-I: Adding /anaconda3/envs/biobb_AMBER_MDsetup_tutorials/dat/leap/parm to search path.
-I: Adding /anaconda3/envs/biobb_AMBER_MDsetup_tutorials/dat/leap/cmd to search path.
-f: Source 47a0068a-6a8f-44cf-b009-eb08c11b5da8/leap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./47a0068a-6a8f-44cf-b009-eb08c11b5da8/leap.in
----- Source: /anaconda3/envs/biobb_AMBER_MDsetup_tutorials/

0

### Visualizing 3D structure
Visualizing the **PDB structure** using **NGL**. <br>
Try to identify the differences between the structure generated for the **system topology** and the **original one** (e.g. hydrogen atoms). 

In [57]:
import nglview
import ipywidgets

# Show protein
view = nglview.show_structure_file(output_pdb_path)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein', opacity='0.4')
view.add_representation(repr_type='ball+stick', selection='protein')
view.add_representation(repr_type='ball+stick', radius='0.5', selection='JZ4')
view._remote_call('setSize', target='Widget', args=['','600px'])

view.render_image()
view.download_image(filename='ngl6.png')

view

NGLWidget()

<img src='ngl6.png'></img>

<a id="minv"></a>
## Energetically minimize the structure

**Energetically minimize** the **protein-ligand complex structure** (in vacuo) using the **sander tool** from the **AMBER MD package**. This step is **relaxing the structure**, usually **constrained**, especially when coming from an X-ray **crystal structure**. <br/>

The **miminization process** is done in two steps:
- [Step 1](#minv_1): **Hydrogen** minimization, applying **position restraints** (50 Kcal/mol.$Å^{2}$) to the **protein heavy atoms**.
- [Step 2](#minv_2): **System** minimization, with **no restraints**.
***
**Building Blocks** used:
 - [sander_mdrun](https://biobb-amber.readthedocs.io/en/latest/sander.html#module-sander.sander_mdrun) from **biobb_amber.sander.sander_mdrun**
 - [process_minout](https://biobb-amber.readthedocs.io/en/latest/process.html#module-process.process_minout) from **biobb_amber.process.process_minout**
***

<a id="minv_1"></a>
### Step 1: Minimize Hydrogens
**Hydrogen** minimization, applying **position restraints** (50 Kcal/mol.$Å^{2}$) to the **protein heavy atoms**.

In [15]:
# Import module
from biobb_amber.sander.sander_mdrun import sander_mdrun

# Create prop dict and inputs/outputs
output_h_min_traj_path = 'sander.h_min.x'
output_h_min_rst_path = 'sander.h_min.rst'
output_h_min_log_path = 'sander.h_min.log'

prop = {
    'simulation_type' : "min_vacuo",
    "mdin" : { 
        'maxcyc' : 500,
        'ntpr' : 5,
        'ntr' : 1,
        'restraintmask' : '\":*&!@H=\"',
        'restraint_wt' : 50.0
    }
}

# Create and launch bb
sander_mdrun(input_top_path=output_top_path,
            input_crd_path=output_crd_path,
            input_ref_path=output_crd_path,
            output_traj_path=output_h_min_traj_path,
            output_rst_path=output_h_min_rst_path,
            output_log_path=output_h_min_log_path,
            properties=prop)

2021-06-22 14:33:22,291 [MainThread  ] [INFO ]  Creating 552ce4cb-1feb-474b-9c35-2661dd87fec7 temporary folder
2021-06-22 14:33:22,293 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-06-22 14:33:46,284 [MainThread  ] [INFO ]  sander -O -i 552ce4cb-1feb-474b-9c35-2661dd87fec7/sander.mdin -p structure.leap.top -c structure.leap.crd -r sander.h_min.rst -o sander.h_min.log -x sander.h_min.x -ref  structure.leap.crd

2021-06-22 14:33:46,285 [MainThread  ] [INFO ]  Exit code 0

2021-06-22 14:33:46,286 [MainThread  ] [INFO ]  Removed: mdinfo, 552ce4cb-1feb-474b-9c35-2661dd87fec7


0

### Checking Energy Minimization results
Checking **energy minimization** results. Plotting **potential energy** along time during the **minimization process**.

In [16]:
# Import module
from biobb_amber.process.process_minout import process_minout

# Create prop dict and inputs/outputs
output_h_min_dat_path = 'sander.h_min.energy.dat'

prop = {
    "terms" : ['ENERGY']
}

# Create and launch bb
process_minout(input_log_path=output_h_min_log_path,
            output_dat_path=output_h_min_dat_path,
            properties=prop)  

2021-06-22 14:33:46,306 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-06-22 14:33:46,375 [MainThread  ] [INFO ]  process_minout.perl  sander.h_min.log

2021-06-22 14:33:46,376 [MainThread  ] [INFO ]  Exit code 0

2021-06-22 14:33:46,377 [MainThread  ] [INFO ]  Processing sander output file (sander.h_min.log)...
Processing step 50 of a possible 500...
Processing step 100 of a possible 500...
Processing step 150 of a possible 500...
Processing step 200 of a possible 500...
Processing step 250 of a possible 500...
Processing step 300 of a possible 500...
Processing step 350 of a possible 500...
Processing step 400 of a possible 500...
Processing step 450 of a possible 500...
Processing step 500 of a possible 500...
Processing step 500 of a possible 500...
Starting output...
Outputing summary.NSTEP
Outputing summary.ENERGY
Outputing summary.RMS
Outputing summary.GMAX
Outputing summary.NAME
Outputing summary.NUMBER
Outputing summary.BOND
Outputi

0

In [17]:
#Read data from file and filter energy values higher than 1000 Kj/mol^-1
with open(output_h_min_dat_path,'r') as energy_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in energy_file 
            if not line.startswith(("#","@")) 
            if float(line.split()[1]) < 1000 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Energy Minimization",
                        xaxis=dict(title = "Energy Minimization Step"),
                        yaxis=dict(title = "Potential Energy kcal/mol")
                       )
}

plotly.offline.iplot(fig)

<a id="minv_2"></a>
### Step 2: Minimize the system
**System** minimization, with **restraints** only on the **small molecule**, to avoid a possible change in position due to **protein repulsion**.

In [18]:
# Import module
from biobb_amber.sander.sander_mdrun import sander_mdrun

# Create prop dict and inputs/outputs
output_n_min_traj_path = 'sander.n_min.x'
output_n_min_rst_path = 'sander.n_min.rst'
output_n_min_log_path = 'sander.n_min.log'

prop = {
    'simulation_type' : "min_vacuo",
    "mdin" : { 
        'maxcyc' : 500,
        'ntpr' : 5,
        'restraintmask' : '\":' + ligandCode + '\"',
        'restraint_wt' : 500.0
    }
}

# Create and launch bb
sander_mdrun(input_top_path=output_top_path,
            input_crd_path=output_h_min_rst_path,
            output_traj_path=output_n_min_traj_path,
            output_rst_path=output_n_min_rst_path,
            output_log_path=output_n_min_log_path,
            properties=prop)

2021-06-22 14:33:47,029 [MainThread  ] [INFO ]  Creating 7e579700-6ba4-40aa-853c-dafa2ef0f191 temporary folder
2021-06-22 14:33:47,031 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-06-22 14:34:10,096 [MainThread  ] [INFO ]  sander -O -i 7e579700-6ba4-40aa-853c-dafa2ef0f191/sander.mdin -p structure.leap.top -c sander.h_min.rst -r sander.n_min.rst -o sander.n_min.log -x sander.n_min.x

2021-06-22 14:34:10,097 [MainThread  ] [INFO ]  Exit code 0

2021-06-22 14:34:10,098 [MainThread  ] [INFO ]  Removed: mdinfo, 7e579700-6ba4-40aa-853c-dafa2ef0f191


0

### Checking Energy Minimization results
Checking **energy minimization** results. Plotting **potential energy** by time during the **minimization process**.

In [19]:
# Import module
from biobb_amber.process.process_minout import process_minout

# Create prop dict and inputs/outputs
output_n_min_dat_path = 'sander.n_min.energy.dat'

prop = {
    "terms" : ['ENERGY']
}

# Create and launch bb
process_minout(input_log_path=output_n_min_log_path,
            output_dat_path=output_n_min_dat_path,
            properties=prop)

2021-06-22 14:34:10,113 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-06-22 14:34:10,155 [MainThread  ] [INFO ]  process_minout.perl  sander.n_min.log

2021-06-22 14:34:10,156 [MainThread  ] [INFO ]  Exit code 0

2021-06-22 14:34:10,156 [MainThread  ] [INFO ]  Processing sander output file (sander.n_min.log)...
Processing step 50 of a possible 500...
Processing step 100 of a possible 500...
Processing step 150 of a possible 500...
Processing step 200 of a possible 500...
Processing step 250 of a possible 500...
Processing step 300 of a possible 500...
Processing step 350 of a possible 500...
Processing step 400 of a possible 500...
Processing step 450 of a possible 500...
Processing step 500 of a possible 500...
Processing step 500 of a possible 500...
Starting output...
Outputing summary.NSTEP
Outputing summary.ENERGY
Outputing summary.RMS
Outputing summary.GMAX
Outputing summary.NAME
Outputing summary.NUMBER
Outputing summary.BOND
Outputi

0

In [20]:
#Read data from file and filter energy values higher than 1000 Kj/mol^-1
with open(output_n_min_dat_path,'r') as energy_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in energy_file 
            if not line.startswith(("#","@")) 
            if float(line.split()[1]) < 1000 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Energy Minimization",
                        xaxis=dict(title = "Energy Minimization Step"),
                        yaxis=dict(title = "Potential Energy kcal/mol")
                       )
}

plotly.offline.iplot(fig)

<a id="box"></a>
***
## Create solvent box and solvating the system
Define the unit cell for the **protein structure MD system** to fill it with water molecules.<br>
A **truncated octahedron box** is used to define the unit cell, with a **distance from the protein to the box edge of 9.0 Angstroms**. <br>
The solvent type used is the default **TIP3P** water model, a generic 3-point solvent model.
***
**Building Blocks** used:
 - [amber_to_pdb](https://biobb-amber.readthedocs.io/en/latest/ambpdb.html#module-ambpdb.amber_to_pdb) from **biobb_amber.ambpdb.amber_to_pdb**
 - [leap_solvate](https://biobb-amber.readthedocs.io/en/latest/leap.html#module-leap.leap_solvate) from **biobb_amber.leap.leap_solvate** 
***

### Getting minimized structure
Getting the result of the **energetic minimization** and converting it to **PDB format** to be then used as input for the **water box generation**. <br/>This is achieved by converting from **AMBER topology + coordinates** files to a **PDB file** using the **ambpdb** tool from the **AMBER MD package**.

In [21]:
# Import module
from biobb_amber.ambpdb.amber_to_pdb import amber_to_pdb

# Create prop dict and inputs/outputs
output_ambpdb_path = 'structure.ambpdb.pdb'

# Create and launch bb
amber_to_pdb(input_top_path=output_top_path,
            input_crd_path=output_h_min_rst_path,
            output_pdb_path=output_ambpdb_path)

2021-06-22 14:34:10,210 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-06-22 14:34:10,244 [MainThread  ] [INFO ]  ambpdb  -p structure.leap.top -c sander.h_min.rst >  structure.ambpdb.pdb

2021-06-22 14:34:10,245 [MainThread  ] [INFO ]  Exit code 0



0

### Create water box
Define the **unit cell** for the **protein-ligand complex structure MD system** and fill it with **water molecules**.<br/>

In [22]:
# Import module
from biobb_amber.leap.leap_solvate import leap_solvate

# Create prop dict and inputs/outputs
output_solv_pdb_path = 'structure.solv.pdb'
output_solv_top_path = 'structure.solv.parmtop'
output_solv_crd_path = 'structure.solv.crd'

prop = {
    "forcefield" : ["protein.ff14SB","gaff"],
    "water_type": "TIP3PBOX",
    "distance_to_molecule": "9.0",   
    "box_type": "truncated_octahedron"
}

# Create and launch bb
leap_solvate(input_pdb_path=output_ambpdb_path,
             input_lib_path=output_acpype_lib,
             input_frcmod_path=output_acpype_frcmod,
             output_pdb_path=output_solv_pdb_path,
             output_top_path=output_solv_top_path,
             output_crd_path=output_solv_crd_path,
             properties=prop)

2021-06-22 14:34:10,262 [MainThread  ] [INFO ]  Creating 2074c37b-6879-488a-8b34-027ffb15d8e9 temporary folder
2021-06-22 14:34:10,263 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-06-22 14:34:11,240 [MainThread  ] [INFO ]  tleap  -f 2074c37b-6879-488a-8b34-027ffb15d8e9/leap.in

2021-06-22 14:34:11,242 [MainThread  ] [INFO ]  Exit code 0

2021-06-22 14:34:11,243 [MainThread  ] [INFO ]  -I: Adding /anaconda3/envs/biobb_AMBER_MDsetup_tutorials/dat/leap/prep to search path.
-I: Adding /anaconda3/envs/biobb_AMBER_MDsetup_tutorials/dat/leap/lib to search path.
-I: Adding /anaconda3/envs/biobb_AMBER_MDsetup_tutorials/dat/leap/parm to search path.
-I: Adding /anaconda3/envs/biobb_AMBER_MDsetup_tutorials/dat/leap/cmd to search path.
-f: Source 2074c37b-6879-488a-8b34-027ffb15d8e9/leap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./2074c37b-6879-488a-8b34-027ffb15d8e9/leap.in
----- Source: /anaconda3/envs/biobb_AMBER_MDsetup_tutorials/

0

<a id="ions"></a>
## Adding ions

**Neutralizing** the system and adding an additional **ionic concentration** using the **leap tool** from the **AMBER MD package**. <br/>
Using **Sodium (Na+)** and **Chloride (Cl-)** counterions and an **additional ionic concentration** of 150mM.
***
**Building Blocks** used:
 - [leap_add_ions](https://biobb-amber.readthedocs.io/en/latest/leap.html#module-leap.leap_add_ions) from **biobb_amber.leap.leap_add_ions**
***

In [23]:
# Import module
from biobb_amber.leap.leap_add_ions import leap_add_ions

# Create prop dict and inputs/outputs
output_ions_pdb_path = 'structure.ions.pdb'
output_ions_top_path = 'structure.ions.parmtop'
output_ions_crd_path = 'structure.ions.crd'

prop = {
    "forcefield" : ["protein.ff14SB","gaff"],
    "neutralise" : True,
    "positive_ions_type": "Na+",
    "negative_ions_type": "Cl-",
    "ionic_concentration" : 150, # 150mM
    "box_type": "truncated_octahedron"
}

# Create and launch bb
leap_add_ions(input_pdb_path=output_solv_pdb_path,
            input_lib_path=output_acpype_lib,
            input_frcmod_path=output_acpype_frcmod,
           output_pdb_path=output_ions_pdb_path,
           output_top_path=output_ions_top_path,
           output_crd_path=output_ions_crd_path,
           properties=prop)

2021-06-22 14:34:11,288 [MainThread  ] [INFO ]  Creating 350eea0d-4d40-495e-b8ab-eb8664e6e207 temporary folder
2021-06-22 14:34:11,383 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-06-22 14:34:13,439 [MainThread  ] [INFO ]  tleap  -f 350eea0d-4d40-495e-b8ab-eb8664e6e207/leap.in

2021-06-22 14:34:13,440 [MainThread  ] [INFO ]  Exit code 0

2021-06-22 14:34:13,441 [MainThread  ] [INFO ]  -I: Adding /anaconda3/envs/biobb_AMBER_MDsetup_tutorials/dat/leap/prep to search path.
-I: Adding /anaconda3/envs/biobb_AMBER_MDsetup_tutorials/dat/leap/lib to search path.
-I: Adding /anaconda3/envs/biobb_AMBER_MDsetup_tutorials/dat/leap/parm to search path.
-I: Adding /anaconda3/envs/biobb_AMBER_MDsetup_tutorials/dat/leap/cmd to search path.
-f: Source 350eea0d-4d40-495e-b8ab-eb8664e6e207/leap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./350eea0d-4d40-495e-b8ab-eb8664e6e207/leap.in
----- Source: /anaconda3/envs/biobb_AMBER_MDsetup_tutorials/

0

### Visualizing 3D structure
Visualizing the **protein-ligand complex system** with the newly added **solvent box** and **counterions** using **NGL**.<br> Note the **truncated octahedron box** filled with **water molecules** surrounding the **protein structure**, as well as the randomly placed **positive** (Na+, blue) and **negative** (Cl-, gray) **counterions**.

In [58]:
# Show protein
view = nglview.show_structure_file(output_ions_pdb_path)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein')
view.add_representation(repr_type='ball+stick', selection='solvent')
view.add_representation(repr_type='spacefill', selection='Cl- Na+')
view._remote_call('setSize', target='Widget', args=['','600px'])

view.render_image()
view.download_image(filename='ngl7.png')

view

NGLWidget()

<img src='ngl7.png'></img>

<a id="min"></a>
## Energetically minimize the system

**Energetically minimize** the **system** (protein structure + ligand + solvent + ions) using the **sander tool** from the **AMBER MD package**. **Restraining heavy atoms** with a force constant of 15 15 Kcal/mol.$Å^{2}$ to their initial positions.

- [Step 1](#emStep1): Energetically minimize the **system** through 500 minimization cycles.
- [Step 2](#emStep2): Checking **energy minimization** results. Plotting energy by time during the **minimization** process. 
***
**Building Blocks** used:
 - [sander_mdrun](https://biobb-amber.readthedocs.io/en/latest/sander.html#module-sander.sander_mdrun) from **biobb_amber.sander.sander_mdrun**
 - [process_minout](https://biobb-amber.readthedocs.io/en/latest/process.html#module-process.process_minout) from **biobb_amber.process.process_minout**
***

<a id="emStep1"></a>
### Step 1: Running Energy Minimization
The **minimization** type of the **simulation_type property** contains the main default parameters to run an **energy minimization**:

-  imin  = 1 ;    Minimization flag, perform an energy minimization.
-  maxcyc = 500;  The maximum number of cycles of minimization.
-  ntb = 1;       Periodic boundaries: constant volume.
-  ntmin = 2;     Minimization method: steepest descent.


In this particular example, the method used to run the **energy minimization** is the default **steepest descent**, with a **maximum number of 500 cycles** and **periodic conditions**.

In [25]:
# Import module
from biobb_amber.sander.sander_mdrun import sander_mdrun

# Create prop dict and inputs/outputs
output_min_traj_path = 'sander.min.x'
output_min_rst_path = 'sander.min.rst'
output_min_log_path = 'sander.min.log'

prop = {
    "simulation_type" : "minimization",
    "mdin" : { 
        'maxcyc' : 300, # Reducing the number of minimization steps for the sake of time
        'ntr' : 1,      # Overwritting restrain parameter
        'restraintmask' : '\"!:WAT,Cl-,Na+\"',      # Restraining solute
        'restraint_wt' : 15.0                       # With a force constant of 50 Kcal/mol*A2
    }
}

# Create and launch bb
sander_mdrun(input_top_path=output_ions_top_path,
            input_crd_path=output_ions_crd_path,
            input_ref_path=output_ions_crd_path,
            output_traj_path=output_min_traj_path,
            output_rst_path=output_min_rst_path,
            output_log_path=output_min_log_path,
            properties=prop)

2021-06-22 14:34:13,736 [MainThread  ] [INFO ]  Creating 7a502ae4-ffcb-462e-800c-38f0dceb6f42 temporary folder
2021-06-22 14:34:13,737 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-06-22 14:36:10,064 [MainThread  ] [INFO ]  sander -O -i 7a502ae4-ffcb-462e-800c-38f0dceb6f42/sander.mdin -p structure.ions.parmtop -c structure.ions.crd -r sander.min.rst -o sander.min.log -x sander.min.x -ref  structure.ions.crd

2021-06-22 14:36:10,065 [MainThread  ] [INFO ]  Exit code 0

2021-06-22 14:36:10,067 [MainThread  ] [INFO ]  Removed: mdinfo, 7a502ae4-ffcb-462e-800c-38f0dceb6f42


0

<a id="emStep2"></a>
### Step 2: Checking Energy Minimization results
Checking **energy minimization** results. Plotting **potential energy** along time during the **minimization process**.

In [26]:
# Import module
from biobb_amber.process.process_minout import process_minout

# Create prop dict and inputs/outputs
output_dat_path = 'sander.min.energy.dat'

prop = {
    "terms" : ['ENERGY']
}

# Create and launch bb
process_minout(input_log_path=output_min_log_path,
            output_dat_path=output_dat_path,
            properties=prop)

2021-06-22 14:36:10,081 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-06-22 14:36:10,099 [MainThread  ] [INFO ]  process_minout.perl  sander.min.log

2021-06-22 14:36:10,101 [MainThread  ] [INFO ]  Exit code 0

2021-06-22 14:36:10,102 [MainThread  ] [INFO ]  Processing sander output file (sander.min.log)...
Processing step 150 of a possible 300...
Processing step 300 of a possible 300...
Processing step 300 of a possible 300...
Starting output...
Outputing summary.NSTEP
Outputing summary.ENERGY
Outputing summary.RMS
Outputing summary.GMAX
Outputing summary.NAME
Outputing summary.NUMBER
Outputing summary.BOND
Outputing summary.ANGLE
Outputing summary.DIHEDRAL
Outputing summary.VDWAALS
Outputing summary.EEL
Outputing summary.HBOND
Outputing summary.VDW14
Outputing summary.EEL14
Outputing summary.RESTRAINT

2021-06-22 14:36:10,107 [MainThread  ] [INFO ]  Removed: [PosixPath('summary.DIHEDRAL'), PosixPath('summary.NUMBER'), PosixPath('summary.B

0

In [27]:
#Read data from file and filter energy values higher than 1000 Kj/mol^-1
with open(output_dat_path,'r') as energy_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in energy_file 
            if not line.startswith(("#","@")) 
            if float(line.split()[1]) < 1000 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Energy Minimization",
                        xaxis=dict(title = "Energy Minimization Step"),
                        yaxis=dict(title = "Potential Energy kcal/mol")
                       )
}

plotly.offline.iplot(fig)

<a id="heating"></a>
## Heating the system

**Warming up** the **prepared system** using the **sander tool** from the **AMBER MD package**. Going from 0 to the desired **temperature**, in this particular example, 300K. **Solute atoms restrained** (force constant of 10 Kcal/mol). Length 5ps.
***
- [Step 1](#heatStep1): Warming up the **system** through 500 MD steps.
- [Step 2](#heatStep2): Checking results for the **system warming up**. Plotting **temperature** along time during the **heating** process. 
***
**Building Blocks** used:
 - [sander_mdrun](https://biobb-amber.readthedocs.io/en/latest/sander.html#module-sander.sander_mdrun) from **biobb_amber.sander.sander_mdrun**
 - [process_mdout](https://biobb-amber.readthedocs.io/en/latest/process.html#module-process.process_mdout) from **biobb_amber.process.process_mdout**
***

<a id="heatStep1"></a>
### Step 1: Warming up the system
The **heat** type of the **simulation_type property** contains the main default parameters to run a **system warming up**:

-  imin = 0;&nbsp;&nbsp;&nbsp;      Run MD (no minimization)
-  ntx = 5;&nbsp;&nbsp;&nbsp;       Read initial coords and vels from restart file
-  cut = 10.0;&nbsp;&nbsp;&nbsp;    Cutoff for non bonded interactions in Angstroms
-  ntr = 0;&nbsp;&nbsp;&nbsp;       No restrained atoms
-  ntc = 2;&nbsp;&nbsp;&nbsp;       SHAKE for constraining length of bonds involving Hydrogen atoms
-  ntf = 2;&nbsp;&nbsp;&nbsp;       Bond interactions involving H omitted
-  ntt = 3;&nbsp;&nbsp;&nbsp;       Constant temperature using Langevin dynamics
-  ig = -1;&nbsp;&nbsp;&nbsp;       Seed for pseudo-random number generator
-  ioutfm = 1;&nbsp;&nbsp;&nbsp;    Write trajectory in netcdf format
-  iwrap = 1;&nbsp;&nbsp;&nbsp;     Wrap coords into primary box
-  nstlim = 5000;&nbsp;&nbsp;&nbsp; Number of MD steps 
-  dt = 0.002;&nbsp;&nbsp;&nbsp;    Time step (in ps)
-  tempi = 0.0;&nbsp;&nbsp;&nbsp;   Initial temperature (0 K)
-  temp0 = 300.0;&nbsp;&nbsp;&nbsp; Final temperature (300 K)
-  irest = 0;&nbsp;&nbsp;&nbsp;     No restart from previous simulation
-  ntb = 1;&nbsp;&nbsp;&nbsp;       Periodic boundary conditions at constant volume
-  gamma_ln = 1.0;&nbsp;&nbsp;&nbsp;   Collision frequency for Langevin dynamics (in 1/ps)

In this particular example, the **heating** of the system is done in **2500 steps** (5ps) and is going **from 0K to 300K** (note that the number of steps has been reduced in this tutorial, for the sake of time). 

In [28]:
# Import module
from biobb_amber.sander.sander_mdrun import sander_mdrun

# Create prop dict and inputs/outputs
output_heat_traj_path = 'sander.heat.netcdf'
output_heat_rst_path = 'sander.heat.rst'
output_heat_log_path = 'sander.heat.log'

prop = {
    "simulation_type" : "heat",
    "mdin" : { 
        'nstlim' : 2500, # Reducing the number of steps for the sake of time (5ps)
        'ntr' : 1,     # Overwritting restrain parameter
        'restraintmask' : '\"!:WAT,Cl-,Na+\"',      # Restraining solute
        'restraint_wt' : 10.0                       # With a force constant of 10 Kcal/mol*A2
    }
}

# Create and launch bb
sander_mdrun(input_top_path=output_ions_top_path,
            input_crd_path=output_min_rst_path,
            input_ref_path=output_min_rst_path,
            output_traj_path=output_heat_traj_path,
            output_rst_path=output_heat_rst_path,
            output_log_path=output_heat_log_path,
            properties=prop)

2021-06-22 14:36:10,186 [MainThread  ] [INFO ]  Creating 4dfd3a1c-b371-4152-8ac1-b7a85c976396 temporary folder
2021-06-22 14:36:10,189 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-06-22 14:57:48,864 [MainThread  ] [INFO ]  sander -O -i 4dfd3a1c-b371-4152-8ac1-b7a85c976396/sander.mdin -p structure.ions.parmtop -c sander.min.rst -r sander.heat.rst -o sander.heat.log -x sander.heat.netcdf -ref  sander.min.rst

2021-06-22 14:57:48,865 [MainThread  ] [INFO ]  Exit code 0

2021-06-22 14:57:48,866 [MainThread  ] [INFO ]  Removed: mdinfo, 4dfd3a1c-b371-4152-8ac1-b7a85c976396


0

<a id="heatStep2"></a>
### Step 2: Checking results from the system warming up
Checking **system warming up** output. Plotting **temperature** along time during the **heating process**.

In [29]:
# Import module
from biobb_amber.process.process_mdout import process_mdout

# Create prop dict and inputs/outputs
output_dat_heat_path = 'sander.md.temp.dat'

prop = {
    "terms" : ['TEMP']
}

# Create and launch bb
process_mdout(input_log_path=output_heat_log_path,
            output_dat_path=output_dat_heat_path,
            properties=prop)

2021-06-22 14:57:48,883 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-06-22 14:57:48,912 [MainThread  ] [INFO ]  process_mdout.perl  sander.heat.log

2021-06-22 14:57:48,914 [MainThread  ] [INFO ]  Exit code 0

2021-06-22 14:57:48,915 [MainThread  ] [INFO ]  Processing sander output file (sander.heat.log)...
Starting output...
Outputing summary.TEMP
Outputing summary_avg.TEMP
Outputing summary_rms.TEMP
Outputing summary.TSOLUTE
Outputing summary_avg.TSOLUTE
Outputing summary_rms.TSOLUTE
Outputing summary.TSOLVENT
Outputing summary_avg.TSOLVENT
Outputing summary_rms.TSOLVENT
Outputing summary.PRES
Outputing summary_avg.PRES
Outputing summary_rms.PRES
Outputing summary.EKCMT
Outputing summary_avg.EKCMT
Outputing summary_rms.EKCMT
Outputing summary.ETOT
Outputing summary_avg.ETOT
Outputing summary_rms.ETOT
Outputing summary.EKTOT
Outputing summary_avg.EKTOT
Outputing summary_rms.EKTOT
Outputing summary.EPTOT
Outputing summary_avg.EPTOT
Outputi

0

In [30]:
#Read data from file and filter energy values higher than 1000 Kj/mol^-1
with open(output_dat_heat_path,'r') as energy_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in energy_file 
            if not line.startswith(("#","@")) 
            if float(line.split()[1]) < 1000 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Heating process",
                        xaxis=dict(title = "Heating Step (ps)"),
                        yaxis=dict(title = "Temperature (K)")
                       )
}

plotly.offline.iplot(fig)

<a id="nvt"></a>
***
## Equilibrate the system (NVT)
Equilibrate the **protein-ligand complex system** in **NVT ensemble** (constant Number of particles, Volume and Temperature). Protein **heavy atoms** will be restrained using position restraining forces: movement is permitted, but only after overcoming a substantial energy penalty. The utility of position restraints is that they allow us to equilibrate our solvent around our protein, without the added variable of structural changes in the protein.

- [Step 1](#eqNVTStep1): Equilibrate the **protein system** with **NVT** ensemble.
- [Step 2](#eqNVTStep2): Checking **NVT Equilibration** results. Plotting **system temperature** by time during the **NVT equilibration** process.  
***
**Building Blocks** used:
 - [sander_mdrun](https://biobb-amber.readthedocs.io/en/latest/sander.html#module-sander.sander_mdrun) from **biobb_amber.sander.sander_mdrun**
 - [process_mdout](https://biobb-amber.readthedocs.io/en/latest/process.html#module-process.process_mdout) from **biobb_amber.process.process_mdout** 
***

<a id="eqNVTStep1"></a>
### Step 1: Equilibrating the system (NVT)
The **nvt** type of the **simulation_type property** contains the main default parameters to run a **system equilibration in NVT ensemble**:

-  imin = 0;&nbsp;&nbsp;&nbsp;      Run MD (no minimization)
-  ntx = 5;&nbsp;&nbsp;&nbsp;       Read initial coords and vels from restart file
-  cut = 10.0;&nbsp;&nbsp;&nbsp;    Cutoff for non bonded interactions in Angstroms
-  ntr = 0;&nbsp;&nbsp;&nbsp;       No restrained atoms
-  ntc = 2;&nbsp;&nbsp;&nbsp;       SHAKE for constraining length of bonds involving Hydrogen atoms
-  ntf = 2;&nbsp;&nbsp;&nbsp;       Bond interactions involving H omitted
-  ntt = 3;&nbsp;&nbsp;&nbsp;       Constant temperature using Langevin dynamics
-  ig = -1;&nbsp;&nbsp;&nbsp;       Seed for pseudo-random number generator
-  ioutfm = 1;&nbsp;&nbsp;&nbsp;    Write trajectory in netcdf format
-  iwrap = 1;&nbsp;&nbsp;&nbsp;     Wrap coords into primary box
-  nstlim = 5000;&nbsp;&nbsp;&nbsp; Number of MD steps 
-  dt = 0.002;&nbsp;&nbsp;&nbsp;    Time step (in ps)
-  irest = 1;&nbsp;&nbsp;&nbsp;     Restart previous simulation
-  ntb = 1;&nbsp;&nbsp;&nbsp;       Periodic boundary conditions at constant volume
-  gamma_ln = 5.0;&nbsp;&nbsp;&nbsp;   Collision frequency for Langevin dynamics (in 1/ps)

In this particular example, the **NVT equilibration** of the system is done in **500 steps** (note that the number of steps has been reduced in this tutorial, for the sake of time). 

In [31]:
# Import module
from biobb_amber.sander.sander_mdrun import sander_mdrun

# Create prop dict and inputs/outputs
output_nvt_traj_path = 'sander.nvt.netcdf'
output_nvt_rst_path = 'sander.nvt.rst'
output_nvt_log_path = 'sander.nvt.log'

prop = {
    "simulation_type" : 'nvt',
    "mdin" : { 
        'nstlim' : 500, # Reducing the number of steps for the sake of time (1ps)
        'ntr' : 1,     # Overwritting restrain parameter
        'restraintmask' : '\"!:WAT,Cl-,Na+ & !@H=\"',      # Restraining solute heavy atoms
        'restraint_wt' : 5.0                              # With a force constant of 5 Kcal/mol*A2
    }
}

# Create and launch bb
sander_mdrun(input_top_path=output_ions_top_path,
            input_crd_path=output_heat_rst_path,
            input_ref_path=output_heat_rst_path,
            output_traj_path=output_nvt_traj_path,
            output_rst_path=output_nvt_rst_path,
            output_log_path=output_nvt_log_path,
            properties=prop)

2021-06-22 14:57:48,980 [MainThread  ] [INFO ]  Creating 071fcbc9-4a22-4537-a033-0c91c57cecd8 temporary folder
2021-06-22 14:57:48,981 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-06-22 15:02:12,764 [MainThread  ] [INFO ]  sander -O -i 071fcbc9-4a22-4537-a033-0c91c57cecd8/sander.mdin -p structure.ions.parmtop -c sander.heat.rst -r sander.nvt.rst -o sander.nvt.log -x sander.nvt.netcdf -ref  sander.heat.rst

2021-06-22 15:02:12,765 [MainThread  ] [INFO ]  Exit code 0

2021-06-22 15:02:12,767 [MainThread  ] [INFO ]  Removed: mdinfo, 071fcbc9-4a22-4537-a033-0c91c57cecd8


0

<a id="eqNVTStep2"></a>
### Step 2: Checking NVT Equilibration results
Checking **NVT Equilibration** results. Plotting **system temperature** by time during the NVT equilibration process. 

In [32]:
# Import module
from biobb_amber.process.process_mdout import process_mdout

# Create prop dict and inputs/outputs
output_dat_nvt_path = 'sander.md.nvt.temp.dat'

prop = {
    "terms" : ['TEMP']
}

# Create and launch bb
process_mdout(input_log_path=output_nvt_log_path,
            output_dat_path=output_dat_nvt_path,
            properties=prop)

2021-06-22 15:02:12,786 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-06-22 15:02:12,805 [MainThread  ] [INFO ]  process_mdout.perl  sander.nvt.log

2021-06-22 15:02:12,807 [MainThread  ] [INFO ]  Exit code 0

2021-06-22 15:02:12,808 [MainThread  ] [INFO ]  Processing sander output file (sander.nvt.log)...
Starting output...
Outputing summary.TEMP
Outputing summary_avg.TEMP
Outputing summary_rms.TEMP
Outputing summary.TSOLUTE
Outputing summary_avg.TSOLUTE
Outputing summary_rms.TSOLUTE
Outputing summary.TSOLVENT
Outputing summary_avg.TSOLVENT
Outputing summary_rms.TSOLVENT
Outputing summary.PRES
Outputing summary_avg.PRES
Outputing summary_rms.PRES
Outputing summary.EKCMT
Outputing summary_avg.EKCMT
Outputing summary_rms.EKCMT
Outputing summary.ETOT
Outputing summary_avg.ETOT
Outputing summary_rms.ETOT
Outputing summary.EKTOT
Outputing summary_avg.EKTOT
Outputing summary_rms.EKTOT
Outputing summary.EPTOT
Outputing summary_avg.EPTOT
Outputing

0

In [33]:
#Read data from file and filter energy values higher than 1000 Kj/mol^-1
with open(output_dat_nvt_path,'r') as energy_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in energy_file 
            if not line.startswith(("#","@")) 
            if float(line.split()[1]) < 1000 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="NVT equilibration",
                        xaxis=dict(title = "Equilibration Step (ps)"),
                        yaxis=dict(title = "Temperature (K)")
                       )
}

plotly.offline.iplot(fig)

<a id="npt"></a>
***
## Equilibrate the system (NPT)
Equilibrate the **protein-ligand complex system** in **NPT ensemble** (constant Number of particles, Pressure and Temperature). Protein **heavy atoms** will be restrained using position restraining forces: movement is permitted, but only after overcoming a substantial energy penalty. The utility of position restraints is that they allow us to equilibrate our solvent around our protein, without the added variable of structural changes in the protein.

- [Step 1](#eqNPTStep1): Equilibrate the **protein system** with **NPT** ensemble.
- [Step 2](#eqNPTStep2): Checking **NPT Equilibration** results. Plotting **system pressure and density** by time during the **NVT equilibration** process.  
***
**Building Blocks** used:
 - [sander_mdrun](https://biobb-amber.readthedocs.io/en/latest/sander.html#module-sander.sander_mdrun) from **biobb_amber.sander.sander_mdrun**
 - [process_mdout](https://biobb-amber.readthedocs.io/en/latest/process.html#module-process.process_mdout) from **biobb_amber.process.process_mdout** 
***

<a id="eqNPTStep1"></a>
### Step 1: Equilibrating the system (NPT)
The **npt** type of the **simulation_type property** contains the main default parameters to run a **system equilibration in NPT ensemble**:

-  imin = 0;&nbsp;&nbsp;&nbsp;      Run MD (no minimization)
-  ntx = 5;&nbsp;&nbsp;&nbsp;       Read initial coords and vels from restart file
-  cut = 10.0;&nbsp;&nbsp;&nbsp;    Cutoff for non bonded interactions in Angstroms
-  ntr = 0;&nbsp;&nbsp;&nbsp;       No restrained atoms
-  ntc = 2;&nbsp;&nbsp;&nbsp;       SHAKE for constraining length of bonds involving Hydrogen atoms
-  ntf = 2;&nbsp;&nbsp;&nbsp;       Bond interactions involving H omitted
-  ntt = 3;&nbsp;&nbsp;&nbsp;       Constant temperature using Langevin dynamics
-  ig = -1;&nbsp;&nbsp;&nbsp;       Seed for pseudo-random number generator
-  ioutfm = 1;&nbsp;&nbsp;&nbsp;    Write trajectory in netcdf format
-  iwrap = 1;&nbsp;&nbsp;&nbsp;     Wrap coords into primary box
-  nstlim = 5000;&nbsp;&nbsp;&nbsp; Number of MD steps 
-  dt = 0.002;&nbsp;&nbsp;&nbsp;    Time step (in ps)
-  irest = 1;&nbsp;&nbsp;&nbsp;     Restart previous simulation
-  gamma_ln = 5.0;&nbsp;&nbsp;&nbsp;   Collision frequency for Langevin dynamics (in 1/ps)
-  pres0 = 1.0;&nbsp;&nbsp;&nbsp;   Reference pressure
-  ntp = 1;&nbsp;&nbsp;&nbsp;       Constant pressure dynamics: md with isotropic position scaling
-  taup = 2.0;&nbsp;&nbsp;&nbsp;    Pressure relaxation time (in ps)

In this particular example, the **NPT equilibration** of the system is done in **500 steps** (note that the number of steps has been reduced in this tutorial, for the sake of time). 

In [34]:
# Import module
from biobb_amber.sander.sander_mdrun import sander_mdrun

# Create prop dict and inputs/outputs
output_npt_traj_path = 'sander.npt.netcdf'
output_npt_rst_path = 'sander.npt.rst'
output_npt_log_path = 'sander.npt.log'

prop = {
    "simulation_type" : 'npt',
    "mdin" : { 
        'nstlim' : 500, # Reducing the number of steps for the sake of time (1ps)
        'ntr' : 1,     # Overwritting restrain parameter
        'restraintmask' : '\"!:WAT,Cl-,Na+ & !@H=\"',      # Restraining solute heavy atoms
        'restraint_wt' : 2.5                               # With a force constant of 2.5 Kcal/mol*A2
    }
}

# Create and launch bb
sander_mdrun(input_top_path=output_ions_top_path,
            input_crd_path=output_nvt_rst_path,
            input_ref_path=output_nvt_rst_path,
            output_traj_path=output_npt_traj_path,
            output_rst_path=output_npt_rst_path,
            output_log_path=output_npt_log_path,
            properties=prop)

2021-06-22 15:02:12,874 [MainThread  ] [INFO ]  Creating 3607b942-e34a-437e-9a93-b6484675b35a temporary folder
2021-06-22 15:02:12,876 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-06-22 15:06:40,892 [MainThread  ] [INFO ]  sander -O -i 3607b942-e34a-437e-9a93-b6484675b35a/sander.mdin -p structure.ions.parmtop -c sander.nvt.rst -r sander.npt.rst -o sander.npt.log -x sander.npt.netcdf -ref  sander.nvt.rst

2021-06-22 15:06:40,893 [MainThread  ] [INFO ]  Exit code 0

2021-06-22 15:06:40,894 [MainThread  ] [INFO ]  Removed: mdinfo, 3607b942-e34a-437e-9a93-b6484675b35a


0

<a id="eqNPTStep2"></a>
### Step 2: Checking NPT Equilibration results
Checking **NPT Equilibration** results. Plotting **system pressure and density** by time during the **NPT equilibration** process. 

In [35]:
# Import module
from biobb_amber.process.process_mdout import process_mdout

# Create prop dict and inputs/outputs
output_dat_npt_path = 'sander.md.npt.dat'

prop = {
    "terms" : ['PRES','DENSITY']
}

# Create and launch bb
process_mdout(input_log_path=output_npt_log_path,
            output_dat_path=output_dat_npt_path,
            properties=prop)

2021-06-22 15:06:40,914 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-06-22 15:06:40,933 [MainThread  ] [INFO ]  process_mdout.perl  sander.npt.log

2021-06-22 15:06:40,935 [MainThread  ] [INFO ]  Exit code 0

2021-06-22 15:06:40,936 [MainThread  ] [INFO ]  Processing sander output file (sander.npt.log)...
Starting output...
Outputing summary.TEMP
Outputing summary_avg.TEMP
Outputing summary_rms.TEMP
Outputing summary.TSOLUTE
Outputing summary_avg.TSOLUTE
Outputing summary_rms.TSOLUTE
Outputing summary.TSOLVENT
Outputing summary_avg.TSOLVENT
Outputing summary_rms.TSOLVENT
Outputing summary.PRES
Outputing summary_avg.PRES
Outputing summary_rms.PRES
Outputing summary.EKCMT
Outputing summary_avg.EKCMT
Outputing summary_rms.EKCMT
Outputing summary.ETOT
Outputing summary_avg.ETOT
Outputing summary_rms.ETOT
Outputing summary.EKTOT
Outputing summary_avg.EKTOT
Outputing summary_rms.EKTOT
Outputing summary.EPTOT
Outputing summary_avg.EPTOT
Outputing

0

In [36]:
# Read pressure and density data from file 
with open(output_dat_npt_path,'r') as pd_file:
    x,y,z = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]),float(line.split()[2]))
            for line in pd_file 
            if not line.startswith(("#","@")) 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

trace1 = go.Scatter(
    x=x,y=y
)
trace2 = go.Scatter(
    x=x,y=z
)

fig = subplots.make_subplots(rows=1, cols=2, print_grid=False)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)

fig['layout']['xaxis1'].update(title='Time (ps)')
fig['layout']['xaxis2'].update(title='Time (ps)')
fig['layout']['yaxis1'].update(title='Pressure (bar)')
fig['layout']['yaxis2'].update(title='Density (Kg*m^-3)')

fig['layout'].update(title='Pressure and Density during NPT Equilibration')
fig['layout'].update(showlegend=False)

plotly.offline.iplot(fig)

<a id="free"></a>
***
## Free Molecular Dynamics Simulation
Upon completion of the **two equilibration phases (NVT and NPT)**, the system is now well-equilibrated at the desired temperature and pressure. The **position restraints** can now be released. The last step of the **protein** MD setup is a short, **free MD simulation**, to ensure the robustness of the system. 
- [Step 1](#mdStep1): Run short MD simulation of the **protein system**.
- [Step 2](#mdStep2): Checking results for the final step of the setup process, the **free MD run**. Plotting **Root Mean Square deviation (RMSd)** and **Radius of Gyration (Rgyr)** by time during the **free MD run** step.
***
**Building Blocks** used:
**Building Blocks** used:
 - [sander_mdrun](https://biobb-amber.readthedocs.io/en/latest/sander.html#module-sander.sander_mdrun) from **biobb_amber.sander.sander_mdrun**
 - [process_mdout](https://biobb-amber.readthedocs.io/en/latest/process.html#module-process.process_mdout) from **biobb_amber.process.process_mdout** 
 - [cpptraj_rms](https://biobb-analysis.readthedocs.io/en/latest/ambertools.html#module-ambertools.cpptraj_rms) from **biobb_analysis.cpptraj.cpptraj_rms**
 - [cpptraj_rgyr](https://biobb-analysis.readthedocs.io/en/latest/ambertools.html#module-ambertools.cpptraj_rgyr) from **biobb_analysis.cpptraj.cpptraj_rgyr**
***

<a id="mdStep1"></a>
### Step 1: Creating portable binary run file to run a free MD simulation

The **free** type of the **simulation_type property** contains the main default parameters to run an **unrestrained MD simulation**:

-  imin = 0;&nbsp;&nbsp;&nbsp;      Run MD (no minimization)
-  ntx = 5;&nbsp;&nbsp;&nbsp;       Read initial coords and vels from restart file
-  cut = 10.0;&nbsp;&nbsp;&nbsp;    Cutoff for non bonded interactions in Angstroms
-  ntr = 0;&nbsp;&nbsp;&nbsp;       No restrained atoms
-  ntc = 2;&nbsp;&nbsp;&nbsp;       SHAKE for constraining length of bonds involving Hydrogen atoms
-  ntf = 2;&nbsp;&nbsp;&nbsp;       Bond interactions involving H omitted
-  ntt = 3;&nbsp;&nbsp;&nbsp;       Constant temperature using Langevin dynamics
-  ig = -1;&nbsp;&nbsp;&nbsp;       Seed for pseudo-random number generator
-  ioutfm = 1;&nbsp;&nbsp;&nbsp;    Write trajectory in netcdf format
-  iwrap = 1;&nbsp;&nbsp;&nbsp;     Wrap coords into primary box
-  nstlim = 5000;&nbsp;&nbsp;&nbsp; Number of MD steps 
-  dt = 0.002;&nbsp;&nbsp;&nbsp;    Time step (in ps)

In this particular example, a short, **5ps-length** simulation (2500 steps) is run, for the sake of time. 

In [37]:
# Import module
from biobb_amber.sander.sander_mdrun import sander_mdrun

# Create prop dict and inputs/outputs
output_free_traj_path = 'sander.free.netcdf'
output_free_rst_path = 'sander.free.rst'
output_free_log_path = 'sander.free.log'

prop = {
    "simulation_type" : 'free',
    "mdin" : { 
        'nstlim' : 2500, # Reducing the number of steps for the sake of time (5ps)
        'ntwx' : 500  # Print coords to trajectory every 500 steps (1 ps)
    }
}

# Create and launch bb
sander_mdrun(input_top_path=output_ions_top_path,
            input_crd_path=output_npt_rst_path,
            output_traj_path=output_free_traj_path,
            output_rst_path=output_free_rst_path,
            output_log_path=output_free_log_path,
            properties=prop)

2021-06-22 15:06:41,080 [MainThread  ] [INFO ]  Creating 4fa1528a-9211-4a59-9648-8840346dfc54 temporary folder
2021-06-22 15:06:41,081 [MainThread  ] [INFO ]  Creating command line with instructions and required arguments
2021-06-22 15:28:10,365 [MainThread  ] [INFO ]  sander -O -i 4fa1528a-9211-4a59-9648-8840346dfc54/sander.mdin -p structure.ions.parmtop -c sander.npt.rst -r sander.free.rst -o sander.free.log -x sander.free.netcdf

2021-06-22 15:28:10,366 [MainThread  ] [INFO ]  Exit code 0

2021-06-22 15:28:10,368 [MainThread  ] [INFO ]  Removed: mdinfo, 4fa1528a-9211-4a59-9648-8840346dfc54


0

<a id="mdStep2"></a>
### Step 2: Checking free MD simulation results

Checking results for the final step of the setup process, the **free MD run**. Plotting **Root Mean Square deviation (RMSd)** and **Radius of Gyration (Rgyr)** by time during the **free MD run** step. **RMSd** against the **experimental structure** (input structure of the pipeline) and against the **minimized and equilibrated structure** (output structure of the NPT equilibration step).

In [38]:
# cpptraj_rms: Computing Root Mean Square deviation to analyse structural stability 
#              RMSd against minimized and equilibrated snapshot (backbone atoms)   

# Import module
from biobb_analysis.ambertools.cpptraj_rms import cpptraj_rms

# Create prop dict and inputs/outputs
output_rms_first = pdbCode+'_rms_first.dat'

prop = {
    'mask': 'backbone',
    'reference': 'first'
}

# Create and launch bb
cpptraj_rms(input_top_path=output_ions_top_path,
            input_traj_path=output_free_traj_path,
            output_cpptraj_path=output_rms_first,
            properties=prop)

2021-06-22 15:28:10,393 [MainThread  ] [INFO ]  Not using any container
2021-06-22 15:28:10,595 [MainThread  ] [INFO ]  cpptraj -i 40846a97-ce16-492a-a940-328006a3bb39/instructions.in

2021-06-22 15:28:10,596 [MainThread  ] [INFO ]  Exit code 0

2021-06-22 15:28:10,596 [MainThread  ] [INFO ]  
CPPTRAJ: Trajectory Analysis. V4.25.6
    ___  ___  ___  ___
     | \/ | \/ | \/ | 
    _|_/\_|_/\_|_/\_|_

| Date/time: 06/22/21 15:28:10
| Available memory: 977.474 MB

INPUT: Reading input from '40846a97-ce16-492a-a940-328006a3bb39/instructions.in'
  [parm structure.ions.parmtop]
	Reading 'structure.ions.parmtop' as Amber Topology
	Radius Set: modified Bondi radii (mbondi)
  [trajin sander.free.netcdf 1 -1 1]
	Reading 'sander.free.netcdf' as Amber NetCDF
  [center !@H*,1H*,2H*,3H* origin]
    CENTER: Centering coordinates using geometric center of atoms in mask (!@H*,1H*,2H*,3H*) to
	coordinate origin.
  [autoimage]
    AUTOIMAGE: To box center based on center of mass, anchor is first molecule

0

In [39]:
# cpptraj_rms: Computing Root Mean Square deviation to analyse structural stability 
#              RMSd against experimental structure (backbone atoms)   

# Import module
from biobb_analysis.ambertools.cpptraj_rms import cpptraj_rms

# Create prop dict and inputs/outputs
output_rms_exp = pdbCode+'_rms_exp.dat'

prop = {
    'mask': 'backbone',
    'reference': 'experimental'
}

# Create and launch bb
cpptraj_rms(input_top_path=output_ions_top_path,
            input_traj_path=output_free_traj_path,
            output_cpptraj_path=output_rms_exp,
            input_exp_path=output_pdb_path, 
            properties=prop)

2021-06-22 15:28:10,617 [MainThread  ] [INFO ]  Not using any container
2021-06-22 15:28:10,823 [MainThread  ] [INFO ]  cpptraj -i fdd4b92a-ac4a-464b-99e1-37527ebb9dc6/instructions.in

2021-06-22 15:28:10,824 [MainThread  ] [INFO ]  Exit code 0

2021-06-22 15:28:10,827 [MainThread  ] [INFO ]  
CPPTRAJ: Trajectory Analysis. V4.25.6
    ___  ___  ___  ___
     | \/ | \/ | \/ | 
    _|_/\_|_/\_|_/\_|_

| Date/time: 06/22/21 15:28:10
| Available memory: 968.180 MB

INPUT: Reading input from 'fdd4b92a-ac4a-464b-99e1-37527ebb9dc6/instructions.in'
  [parm structure.ions.parmtop]
	Reading 'structure.ions.parmtop' as Amber Topology
	Radius Set: modified Bondi radii (mbondi)
  [trajin sander.free.netcdf 1 -1 1]
	Reading 'sander.free.netcdf' as Amber NetCDF
  [center !@H*,1H*,2H*,3H* origin]
    CENTER: Centering coordinates using geometric center of atoms in mask (!@H*,1H*,2H*,3H*) to
	coordinate origin.
  [autoimage]
    AUTOIMAGE: To box center based on center of mass, anchor is first molecule

0

In [40]:
# Read RMS vs first snapshot data from file 
with open(output_rms_first,'r') as rms_first_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in rms_first_file 
            if not line.startswith(("#","@")) 
        ])
    )

# Read RMS vs experimental structure data from file 
with open(output_rms_exp,'r') as rms_exp_file:
    x2,y2 = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in rms_exp_file
            if not line.startswith(("#","@")) 
        ])
    )
    
trace1 = go.Scatter(
    x = x,
    y = y,
    name = 'RMSd vs first'
)

trace2 = go.Scatter(
    x = x,
    y = y2,
    name = 'RMSd vs exp'
)

data = [trace1, trace2]

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": data,
    "layout": go.Layout(title="RMSd during free MD Simulation",
                        xaxis=dict(title = "Time (ps)"),
                        yaxis=dict(title = "RMSd (Angstrom)")
                       )
}

plotly.offline.iplot(fig)


In [41]:
# cpptraj_rgyr: Computing Radius of Gyration to measure the protein compactness during the free MD simulation 

# Import module
from biobb_analysis.ambertools.cpptraj_rgyr import cpptraj_rgyr

# Create prop dict and inputs/outputs
output_rgyr = pdbCode+'_rgyr.dat'

prop = {
    'mask': 'backbone'
}

# Create and launch bb
cpptraj_rgyr(input_top_path=output_ions_top_path,
            input_traj_path=output_free_traj_path,
            output_cpptraj_path=output_rgyr,
            properties=prop)


2021-06-22 15:28:10,897 [MainThread  ] [INFO ]  Not using any container
2021-06-22 15:28:11,134 [MainThread  ] [INFO ]  cpptraj -i 847a9d4f-58d6-4fee-a14b-5fab71fff8e8/instructions.in

2021-06-22 15:28:11,136 [MainThread  ] [INFO ]  Exit code 0

2021-06-22 15:28:11,137 [MainThread  ] [INFO ]  
CPPTRAJ: Trajectory Analysis. V4.25.6
    ___  ___  ___  ___
     | \/ | \/ | \/ | 
    _|_/\_|_/\_|_/\_|_

| Date/time: 06/22/21 15:28:10
| Available memory: 965.882 MB

INPUT: Reading input from '847a9d4f-58d6-4fee-a14b-5fab71fff8e8/instructions.in'
  [parm structure.ions.parmtop]
	Reading 'structure.ions.parmtop' as Amber Topology
	Radius Set: modified Bondi radii (mbondi)
  [trajin sander.free.netcdf 1 -1 1]
	Reading 'sander.free.netcdf' as Amber NetCDF
  [center !@H*,1H*,2H*,3H* origin]
    CENTER: Centering coordinates using geometric center of atoms in mask (!@H*,1H*,2H*,3H*) to
	coordinate origin.
  [autoimage]
    AUTOIMAGE: To box center based on center of mass, anchor is first molecule

0

In [42]:
# Read Rgyr data from file 
with open(output_rgyr,'r') as rgyr_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in rgyr_file 
            if not line.startswith(("#","@")) 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Radius of Gyration",
                        xaxis=dict(title = "Time (ps)"),
                        yaxis=dict(title = "Rgyr (Angstrom)")
                       )
}

plotly.offline.iplot(fig)

<a id="post"></a>
***
## Post-processing and Visualizing resulting 3D trajectory
Post-processing and Visualizing the **protein system** MD setup **resulting trajectory** using **NGL**
- [Step 1](#ppStep1): *Imaging* the resulting trajectory, **stripping out water molecules and ions** and **correcting periodicity issues**.
- [Step 2](#ppStep3): Visualizing the *imaged* trajectory using the *dry* structure as a **topology**. 
***
**Building Blocks** used:
 - [cpptraj_image](https://biobb-analysis.readthedocs.io/en/latest/ambertools.html#module-ambertools.cpptraj_image) from **biobb_analysis.cpptraj.cpptraj_image** 
***

<a id="ppStep1"></a>
### Step 1: *Imaging* the resulting trajectory.
Stripping out **water molecules and ions** and **correcting periodicity issues**  

In [43]:
# cpptraj_image: "Imaging" the resulting trajectory
#                Removing water molecules and ions from the resulting structure

# Import module
from biobb_analysis.ambertools.cpptraj_image import cpptraj_image

# Create prop dict and inputs/outputs
output_imaged_traj = pdbCode+'_imaged_traj.trr'

prop = {
    'mask': 'solute',
    'format': 'trr'
}

# Create and launch bb
cpptraj_image(input_top_path=output_ions_top_path,
            input_traj_path=output_free_traj_path,
            output_cpptraj_path=output_imaged_traj,
            properties=prop)

2021-06-22 15:28:11,231 [MainThread  ] [INFO ]  Not using any container
2021-06-22 15:28:11,448 [MainThread  ] [INFO ]  cpptraj -i 53bd0bea-bf45-459b-b93d-6857d9981d61/instructions.in

2021-06-22 15:28:11,449 [MainThread  ] [INFO ]  Exit code 0

2021-06-22 15:28:11,450 [MainThread  ] [INFO ]  
CPPTRAJ: Trajectory Analysis. V4.25.6
    ___  ___  ___  ___
     | \/ | \/ | \/ | 
    _|_/\_|_/\_|_/\_|_

| Date/time: 06/22/21 15:28:11
| Available memory: 953.430 MB

INPUT: Reading input from '53bd0bea-bf45-459b-b93d-6857d9981d61/instructions.in'
  [parm structure.ions.parmtop]
	Reading 'structure.ions.parmtop' as Amber Topology
	Radius Set: modified Bondi radii (mbondi)
  [trajin sander.free.netcdf 1 -1 1]
	Reading 'sander.free.netcdf' as Amber NetCDF
  [center !@H*,1H*,2H*,3H* origin]
    CENTER: Centering coordinates using geometric center of atoms in mask (!@H*,1H*,2H*,3H*) to
	coordinate origin.
  [autoimage]
    AUTOIMAGE: To box center based on center of mass, anchor is first molecule

0

<a id="ppStep2"></a>
### Step 2: Visualizing the generated dehydrated trajectory.
Using the **imaged trajectory** (output of the [Post-processing step 1](#ppStep1)) with the **dry structure** (output of

In [60]:
# Show trajectory
view = nglview.show_simpletraj(nglview.SimpletrajTrajectory(output_imaged_traj, output_ambpdb_path), gui=True)
view.clear_representations()
view.add_representation('cartoon', color='sstruc')
view.add_representation('licorice', selection='JZ4', color='element', radius=1)
view

NGLWidget(max_frame=4)

<img src='traj.gif'></img>

In [61]:
from time import sleep
# range number of frames for the animated gif trajectory
for frame in range(0, 5):
    # set frame to update coordinates
    view.frame = frame
    # make sure to let NGL spending enough time to update coordinates
    sleep(0.5)
    view.download_image(filename='trj_image{}.png'.format(frame))
    # make sure to let NGL spending enough time to render before going to next frame
    sleep(2.0)

In [63]:
import moviepy.editor as mpy

# go to folder where the images are stored
template = './trj_image{}.png'
# get all (sorted) image files
imagefiles = [template.format(str(i)) for i in range(0, 4, 1)]

# make a gif file
frame_per_second = 8
im = mpy.ImageSequenceClip(imagefiles, fps=frame_per_second)
im.write_gif('traj.gif', fps=frame_per_second)

t:   0%|          | 0/4 [00:00<?, ?it/s, now=None]

MoviePy - Building file traj.gif with imageio.


<a id="output"></a>
## Output files

Important **Output files** generated:
 - {{output_ions_pdb_path}}: **System structure** of the MD setup protocol. Structure generated during the MD setup and used in the MD simulation. With hydrogen atoms, solvent box and counterions.  
 - {{output_free_traj_path}}: **Final trajectory** of the MD setup protocol.
 - {{output_free_rst_path}}: **Final checkpoint file**, with information about the state of the simulation. It can be used to **restart** or **continue** a MD simulation.
 - {{output_ions_top_path}}: **Final topology** of the MD system in AMBER Parm7 format.

**Analysis** (MD setup check) output files generated:
 - {{output_rms_first}}: **Root Mean Square deviation (RMSd)** against **minimized and equilibrated structure** of the final **free MD run step**.
 - {{output_rms_exp}}: **Root Mean Square deviation (RMSd)** against **experimental structure** of the final **free MD run step**.
 - {{output_rgyr}}: **Radius of Gyration** of the final **free MD run step** of the **setup pipeline**.
 

***
<a id="questions"></a>

## Questions & Comments

Questions, issues, suggestions and comments are really welcome!

* GitHub issues:
    * [https://github.com/bioexcel/biobb](https://github.com/bioexcel/biobb)

* BioExcel forum:
    * [https://ask.bioexcel.eu/c/BioExcel-Building-Blocks-library](https://ask.bioexcel.eu/c/BioExcel-Building-Blocks-library)
